In [ ]:
# Volatility calculation for a stock 

In [2]:
import pandas as pd
from scipy.stats import skew, kurtosis
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

tickers = ["AAPL", "MSFT", "VRAY", "BBBYQ"]
results = []

for ticker in tickers:
    data = yf.download(ticker, start="2023-05-19", end="2023-08-19", interval="1d")
    returns = data['Close'].pct_change().dropna()
    drawdown = returns - returns.cummax()

    # Compute the risk measures
    measures = {
        'Ticker': ticker,
        'Standard Deviation': returns.std(),
        'Mean Absolute Deviation': returns.mad(),
        'Semi Standard Deviation': returns[returns < 0].std(),
        'First Lower Partial Moment': returns[returns < 0].mean(),
        'Second Lower Partial Moment': returns[returns < 0].apply(lambda x: x**2).mean(),
        'Value at Risk': returns.quantile(0.05),
        'Conditional Value at Risk': returns[returns <= returns.quantile(0.05)].mean(),
        'Skewness': skew(returns),
        'Kurtosis': kurtosis(returns),
        'Ulcer Index': (drawdown**2).mean()**0.5,
        'Average Drawdown': drawdown.mean(),
        'Max Drawdown': drawdown.min(),
        'Tail Gini of Losses': sum(sorted(returns)[:int(len(returns) * 0.05)]) / sum(sorted(returns)),
    }

    # Entropic Value at Risk
    alpha = 0.05
    sorted_returns = sorted(returns)
    threshold_index = int(alpha * len(sorted_returns))
    threshold = sorted_returns[threshold_index]
    tail_mean = sum(sorted_returns[:threshold_index]) / threshold_index
    measures['Entropic Value at Risk'] = threshold + (1 / (1 - alpha)) * tail_mean

    # Add the results to a list
    results.append(measures)

# Create a DataFrame from the results
measures_df = pd.DataFrame(results)

# Set the index to 'Ticker' column and transpose the DataFrame
measures_df.set_index('Ticker', inplace=True)
transposed_measures_df = measures_df.transpose()

# Print the transposed DataFrame
print("Volatility Measures:")
print(transposed_measures_df)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Volatility Measures:
Ticker                              AAPL       MSFT       VRAY     BBBYQ
Standard Deviation              0.011079   0.015581   0.142948  0.092128
Mean Absolute Deviation         0.008173   0.012266   0.084147  0.069202
Semi Standard Deviation         0.008649   0.008625   0.140353  0.030672
First Lower Partial Moment     -0.008174  -0.012650  -0.096948 -0.070457
Second Lower Partial Moment     0.000139   0.000232   0.028618  0.005870
Value at Risk                  -0.015125  -0.023009  -0.219639 -0.096774
Conditional Value at Risk      -0.023980  -0.029477  -0.431833 -0.118915
Skewness                       -1.049355   0.220647  -2.673381  1.292346
Kurtosis                        4.124792   0.1